In [646]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import regularizers
import tensorflow.random
import pandas as pd
import numpy as np 
import matplotlib 
import matplotlib.pyplot as plt
from sklearn import metrics
import math
from math import log
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

# Summary of code

This code provides a way of predicting functions using discrete derivative regressors.
Functions can be predicted either with a neural network or using Facebook Prophet.
The prediction can be done in one of two ways (nonrecursively and recursively).

For the nonrecursive version, the derivative regressors for future values are calculated after making one pass of predictions without any regressors.
Then the future derivatives are calculated using these predictions.

For the nonrecursive version, the future predictions are made iteratively, each time with an additional derivative regressor.
This may improve the accuracy of the derivatives, leading to better predictions each time.

# Utilities

In [614]:
def derivative(xy_coords):
    '''
    Parameters:
        xy_coords (np array): 0th column is x-coordinates, 1st column is y-coordinates
        
    Returns:
        np array where 0th column is x-coordinates and 1st column is the backwards discrete derivative
    '''
    sort = np.argsort(xy_coords[:,0]) 
    xy_coords = xy_coords[sort,:]
    x_coords = xy_coords[:,0]
    y_coords = xy_coords[:,1]
    deriv = [(y_coords[i]-y_coords[i-1])/(x_coords[i]-x_coords[i-1]) for i in range(1,len(xy_coords))]
    return np.column_stack((x_coords[1:], deriv))

In [615]:
# derivative example:
x = [0,1,2,3,4]
y = [0,1,4,9,16]
xy = np.column_stack((x,y))
derivative(xy)

array([[1., 1.],
       [2., 3.],
       [3., 5.],
       [4., 7.]])

In [616]:
def nthderiv(xy_coords_extended, n, original_length):
    '''
    Parameters:
        xy_coords_extended (2-column np array): points to take derivatives of with at least n historical points
        n (int): number of backwards discrete derivatives to take
        original_length (int): number of points to take derivatives of
    
    Returns:
        derivatives (numpy array): the 0th column has x-coordinates,
                                  ith column has the ith backwards discrete derivative
    '''
    
    derivatives = xy_coords_extended[-original_length:, 0]

    for i in range(1,n+1):
        next_deriv = xy_coords_extended[-original_length - i:, [0,-1]]
        for j in range(i):
            next_deriv = derivative(next_deriv)
        derivatives = np.column_stack((derivatives, next_deriv[:,1]))
    return derivatives

In [617]:
# nthderiv example:
x = [0,1,2,3,4,5,6,7]
y = [0,1,8,27,64,125,216,343]
xy = np.column_stack((x,y))
nthderiv(xy, 2, 5)

array([[  3.,  19.,  12.],
       [  4.,  37.,  18.],
       [  5.,  61.,  24.],
       [  6.,  91.,  30.],
       [  7., 127.,  36.]])

In [618]:
def weighted_mape(pred_values, true_values):
    '''
    Parameters:
        pred_values (array): predicted values
        true_values (array): correct future values
        
    Returns:
        Weighted MAPE on holdout
    '''
    abs_differences = np.abs(pred_values - true_values)
    return np.sum(abs_differences)/np.abs(np.sum(true_values))


def holdout_variance_explained(pred_values, true_values):
    '''
    Parameters:
        pred_values (array): predicted values
        true_values (array): correct future values
        
    Returns:
        Percentage of variance explained for holdout
    '''
    pred_length = len(pred_values)
    score = np.sqrt(metrics.mean_squared_error(pred_values[-pred_length:],true_values[-pred_length:]))
    variance_explained = ((true_values[:-pred_length].std()-score)/true_values[:-pred_length].std())*100
    return variance_explained

In [619]:
def variability(my_array):
    '''
    Parameters:
        my_array (np array)
    Returns:
        float: variability of values in my_array
    '''
    my_array = my_array - min(my_array)
    my_array = my_array/max(my_array)
    sum_square_differences = sum([(my_array[i] - my_array[i-1])**2 for i in range(1, len(my_array))])
    return (sum_square_differences/len(my_array))**0.5

# Tensorflow Code

In [620]:
# withhold first k values and take derivatives backwards

In [621]:
def func_on_array(func, x_coords):
    '''
    Parameters:
        func (function): function to apply to x_coordinates
        x_coords (np array): x-coordinates to get function values of
    
    Returns:
        np array of y-values
    '''
    return np.array([func(x) for x in x_coords])

In [623]:
# Creates a neural network with three hidden layers
seed = 5
def forecast(x_train, y_train, x_pred, dim, layers, neurons):
    '''
    Parameters:
        x_train (numpy array): each row is a single input, contains the appropriate derivatives
        y_train (numpy array): k x 1 array of all of the training labels
        x_pred (numpy array): array with row length the same as x, used as input for predictions
        dim (int): number of columns in x
        layers (int): number of layers (excluding output)
        neurons (int): total number of neurons
    
    Returns:
        pred (np array): predicted values for x-coordinates in x_pred
    
    1. Creates a feedforward neural network where each layer is half the size of the previous one
    2. Trains the network based on x_train and y_train
    3. Returns network output when x_pred is put in the network
    '''
    
    tensorflow.random.set_seed(seed)
    
    model = Sequential()
    layer_size = 2**(layers - 1) * neurons//(2**(layers) - 1)
    model.add(Dense(layer_size, input_dim=dim, activation='relu'))
    
    for i in range(1,layers):
        layer_size = 2**(layers - i - 1) * neurons//(2**(layers) - 1)
        model.add(Dense(layer_size, activation='relu')) # Add hidden layer half the size of the previous one
        
    model.add(Dense(1)) # Output
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(x_train,y_train,verbose=0,epochs=512)
    pred = model.predict(x_pred) # generate predicted values (output)
    return np.squeeze(pred)

def nderiv_extrapolation_nonrecursive(func, x_known, x_pred, n_derivs, layers=3, neurons=2000): 
    '''
    Parameters:
        func (function): function to predict
        x_known (numpy array): x-coordinates of training data
        x_pred (numpy array): x-coordinates to predict at
        n_derivs (int): number of derivatives to use
        layers (int): number of layers (excluding output)
        neurons (int): total number of neurons
    
    Returns:
        weighted_mape (float): weighted mape on holdout
    
    1. Predict function values for x_pred
    2. Create array of backwards derivative regressors
    3. Train network again with all but first n_derivs points and predict values for x_pred
    4. Return weighted mape
    '''
    
    y_known = func_on_array(func, x_known)
    y_future = func_on_array(func, x_pred)
    known_length = len(x_known)
    pred_length = len(x_pred)
    
    pred_values = forecast(x_known,y_known, x_pred, 1, layers, neurons)
    
    if (n_derivs == 0):
        return weighted_mape(pred_values, y_future)
    
    xy_all = np.column_stack((np.concatenate((x_known, x_pred)), np.concatenate((y_known, pred_values))))
    all_derivs = nthderiv(xy_all, n_derivs, known_length + pred_length - n_derivs)
    x_derivs = all_derivs[:-pred_length,:]
    x_derivs_pred = all_derivs[-pred_length:,:]
    y_known_truncated = y_known[n_derivs:]

    pred_values = forecast(x_derivs, y_known_truncated, x_derivs_pred, n_derivs+1, layers, neurons) # forecast based on added regressors

    return weighted_mape(pred_values, y_future)

def nderiv_extrapolation_recursive(func, x_known, x_pred, n_derivs, layers=3, neurons=2000): 
    '''
    Parameters:
        func (function): function to predict
        x_known (numpy array): x-coordinates of training data
        x_pred (numpy array): x-coordinates to predict at
        n_derivs (int): number of derivatives to use
        layers (int): number of layers (excluding output)
        neurons (int): total number of neurons
    
    Returns:
        weighted_mape (float): weighted mape on holdout
    
    1. Predict function values for x_pred
    Loop (with an additional derivatve regressor each time):
        1. Create array of backwards derivative regressors
        2. Update predicted values
    3. Train network again with all but first n_derivs points and predict values for x_pred
    4. Return weighted mape
    '''
    
    y_known = func_on_array(func, x_known)
    y_future = func_on_array(func, x_pred)
    known_length = len(x_known)
    pred_length = len(x_pred)
    
    pred_values = forecast(x_known,y_known, x_pred, 1, layers, neurons)
    
    if (n_derivs == 0):
        return weighted_mape(pred_values, y_future)
    
    for deriv in range(1, n_derivs + 1):
        xy_all = np.column_stack((np.concatenate((x_known, x_pred)), np.concatenate((y_known, pred_values))))
        all_derivs = nthderiv(xy_all, deriv, known_length + pred_length - deriv)
        x_derivs = all_derivs[:-pred_length,:]
        x_derivs_pred = all_derivs[-pred_length:,:]
        y_known_truncated = y_known[deriv:]
        pred_values = forecast(x_derivs, y_known_truncated, x_derivs_pred, deriv + 1, layers, neurons)
    
    return weighted_mape(pred_values, y_future)

## Tensorflow example on y = sin(x)

In [625]:
def my_function(a):
    return math.sin(a)

In [626]:
x_known = np.array([i for i in range(200)])
x_pred = np.array([i for i in range(200,220)])
nderiv_extrapolation_recursive(my_function, x_known, x_pred, n_derivs=2)

14.835859696595731

# Prophet Code

In [630]:
def prophet_deriv(df):
    '''
    Parameters:
        df (DataFrame): DataFrame with ['ds'] column (dates) and ['y'] column (values)
    
    Returns:
        diff (DataFrame): DataFrame with ['ds'] column (dates) and ['y'] column (backwards discrete derivatives)
    '''
    diff = pd.DataFrame()
    diff['ds'] = pd.to_datetime(df['ds'])
    diff = diff.iloc[1:]
    
    dates_diff1 = pd.DataFrame(df['ds'][1:])
    dates_diff2 = pd.DataFrame(df['ds'][:-1])
    dates_diff2.index += 1
    dates_diff = (dates_diff1['ds'] - dates_diff2['ds']).dt.days.values
    
    values_diff1 = pd.DataFrame(df['y'][1:])
    values_diff2 = pd.DataFrame(df['y'][:-1])

    values_diff2.index += 1
    values_diff = (values_diff1['y'] - values_diff2['y']).values

    diff['y'] = values_diff/dates_diff
    return diff

def prophet_nthderiv(df, n, original_length):
    '''
    
    Parameters:
        df (DataFrame): DataFrame with ['ds'] column (dates) and ['y'] column (values)
    
    Returns:
        derivatives (DataFrame): DataFrame with ['ds'] column (dates) columns with backwards discrete derivatives
    '''
    df['ds'] = pd.to_datetime(df['ds'])
    derivatives = pd.DataFrame(df['ds'][-original_length:])
    for i in range(1,n+1):
        next_deriv = df.tail(original_length + i)
        for j in range(i):
            next_deriv = prophet_deriv(next_deriv)
        derivatives['derivative_'+str(j+1)] = next_deriv['y']
    return derivatives

In [631]:
# Use to test Prophet on plain functions
def create_df(x, y):
    df = pd.DataFrame()
    df['ds'] = pd.to_datetime(x, unit='D')
    df['y'] = y
    return df

In [634]:
# Example:
x = [0,1,2,3,4,5,6,7]
y = func_on_array(my_function, x)
df = create_df(x,y)
print(prophet_deriv(df))
print(prophet_nthderiv(df,2,5))

          ds         y
1 1970-01-02  0.841471
2 1970-01-03  0.067826
3 1970-01-04 -0.768177
4 1970-01-05 -0.897923
5 1970-01-06 -0.202122
6 1970-01-07  0.679509
7 1970-01-08  0.936402
          ds  derivative_1  derivative_2
3 1970-01-04     -0.768177     -0.836004
4 1970-01-05     -0.897923     -0.129745
5 1970-01-06     -0.202122      0.695801
6 1970-01-07      0.679509      0.881631
7 1970-01-08      0.936402      0.256893


In [612]:
def prophetpredict(training_data, future_dates, regressors, retail_data = True):
    '''
    Parameters:
        training_data (DataFrame): DataFrame with ['ds'] column (dates) and ['y'] column (values) for known values
        future_dates (list, np array, Series, etc.): dates to predict values at
        regressors (DataFrame): regressor values for all points, including points to predict at
        retail_data (bool): set to True if running Prophet on real data
    
    Returns:
        forecast (DataFrame): DataFrame with ['ds'] column (dates) and ['yhat'] column (values) for predicted values
    '''
    future_dates = pd.DataFrame(future_dates)
    horizon = len(future_dates.index)

    if retail_data:
        m = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
        m.add_country_holidays(country_name='US')
        m.train_holiday_names
    else:
        m = Prophet(yearly_seasonality=False, daily_seasonality=False, weekly_seasonality=False)
    
    for col in regressors:
        m.add_regressor(col)
        
    all_input = pd.concat((training_data, regressors.head(training_length)), axis=1)
    m.fit(all_input)
    all_future_input = pd.concat((future_dates, regressors.tail(horizon)), axis=1)
    forecast = m.predict(all_future_input)
    return forecast[['ds', 'yhat']]

In [642]:
def prophetnderivatives_nonrecursive(training_data, future_data, n_derivs, regressors, retail_data=True):
    '''
    Parameters:
        training_data (DataFrame): DataFrame with ['ds'] column (dates) and ['y'] column (values) for known values
        future_dates (list, np array, Series, etc.): dates to predict values at
        regressors (DataFrame): regressor values for all points, including points to predict at
        retail_data (bool): set to True if running Prophet on real data
    
    Returns:
        forecast (DataFrame): DataFrame with ['ds'] column (dates) and ['yhat'] column (values) for predicted values
        
    1. Predict future values for future_dates with economic indicators
    2. Create DataFrame of backwards derivative regressors and economic indicators
    3. Train Prophet again with regressors on all but first n_derivs points and predict values
    4. Return weighted mape
    '''
    training_data['ds'] = pd.to_datetime(training_data['ds'])
    future_data['ds'] = pd.to_datetime(future_data['ds'])
    training_length = len(training_data)
    horizon = len(future_data.index)
    forecast = prophetpredict(training_data, future_data['ds'], regressors, retail_data)
    
    if n_derivs == 0:
        return weighted_mape(forecast['yhat'].values, np.array(future_data['y']))
    
    col_names = [f'derivative_{i+1}' for i in range(n_derivs)]
    
    forecast.rename(columns={'yhat':'y'}, inplace=True)
    forecast.index += training_length
    training_and_forecast = pd.concat((training_data, forecast))

    all_derivatives = prophet_nthderiv(training_and_forecast, n_derivs, len(training_and_forecast.index)-n_derivs).drop('ds', axis=1)
    all_regressors = pd.concat((all_derivatives, regressors.iloc[n_derivs:]), axis=1)
    new_training = training_data.iloc[n_derivs:training_length]
    
    updated_forecast = prophetpredict(new_training, future_data['ds'], all_regressors, retail_data)
    return weighted_mape(updated_forecast['yhat'].values, future_data['y'].tail(horizon).values)

In [643]:
def prophetnderivatives_recursive(training_data, future_data, n_derivs, regressors, retail_data=True):
    '''
    Parameters:
        training_data (DataFrame): DataFrame with ['ds'] column (dates) and ['y'] column (values) for known values
        future_dates (list, np array, Series, etc.): dates to predict values at
        regressors (DataFrame): regressor values for all points, including points to predict at
        retail_data (bool): set to True if running Prophet on real data
    
    Returns:
        forecast (DataFrame): DataFrame with ['ds'] column (dates) and ['yhat'] column (values) for predicted values
    
    
    1. Predict future values for future_dates with economic indicators
    Loop (with an additional derivatve regressor each time):
        1. Create DataFrame of backwards derivative regressors and economic indicators
        2. Update predicted values
    3. Train Prophet again with regressors all but first n_derivs points and predict values
    4. Return weighted mape
    '''
    training_data['ds'] = pd.to_datetime(training_data['ds'])
    future_data['ds'] = pd.to_datetime(future_data['ds'])
    training_length = len(training_data)
    horizon = len(future_data.index)
    forecast = prophetpredict(training_data, future_data['ds'], regressors, retail_data)
    
    if n_derivs == 0:
        return weighted_mape(forecast['yhat'].values, np.array(future_data['y']))

    for deriv in range(1, n_derivs + 1):
        col_names = [f'derivative_{i+1}' for i in range(n_derivs)]
        forecast.rename(columns={'yhat':'y'}, inplace=True)
        forecast.index += training_length
        training_and_forecast = pd.concat((training_data, forecast))

        all_derivatives = prophet_nthderiv(training_and_forecast, deriv, len(training_and_forecast.index)-deriv).drop('ds', axis=1)
        all_regressors = pd.concat((all_derivatives, regressors.iloc[deriv:]), axis=1)
        new_training = training_data.iloc[deriv:training_length]

        forecast = prophetpredict(new_training, future_data['ds'], all_regressors, retail_data)
    return weighted_mape(forecast['yhat'].values, future_data['y'].tail(horizon).values)

# Sales Data Example

In [644]:
salesdata = pd.read_excel('All_Data_Formatted.xlsx', header=1)
salesdata = salesdata.rename({'date':'ds'}, axis=1)
salesdata = salesdata.iloc[84:204, :]
ds = salesdata['ds']
sales = salesdata.iloc[:, 1:13]
indicators = salesdata.iloc[:, 13:]
indicators.reset_index(inplace=True, drop=True)
sales = pd.concat((ds, sales), axis=1)
sales.reset_index(inplace=True, drop=True)

In [645]:
prophet_results = {}
horizon = len(sales.index)//10
training_length = len(sales.index) - horizon

for sale in sales.iloc[:,1:]:
    data = sales[['ds', sale]]
    data = data.rename(columns={sale: 'y'})
    training_data = data.head(training_length)
    future_data = data.tail(horizon)
    for deriv in range(0,5):
        result = prophetnderivatives_recursive(training_data, future_data, deriv, pd.DataFrame())
        print(sale, deriv, result)
        prophet_results[sale, '', deriv] = result
        for indicator in indicators:
            result = prophetnderivatives_recursive(training_data, future_data, deriv, pd.DataFrame(indicators[indicator]))
            print(sale, indicator, deriv, result)
            prophet_results[sale, indicator, deriv] = result

<ipython-input-643-aabf7bbada85>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data['ds'] = pd.to_datetime(training_data['ds'])
<ipython-input-643-aabf7bbada85>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_data['ds'] = pd.to_datetime(future_data['ds'])
16:00:14 - cmdstanpy - INFO - Chain [1] start processing
16:00:14 - cmdstanpy - INFO - Chain [1] done processing
<ipython-input-643-aabf7bbada85>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

RSAFS 0 0.00964486254705542


16:00:16 - cmdstanpy - INFO - Chain [1] done processing
<ipython-input-643-aabf7bbada85>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data['ds'] = pd.to_datetime(training_data['ds'])
<ipython-input-643-aabf7bbada85>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_data['ds'] = pd.to_datetime(future_data['ds'])
16:00:18 - cmdstanpy - INFO - Chain [1] start processing
16:00:18 - cmdstanpy - INFO - Chain [1] done processing


RSAFS break_inflate10 0 0.0047503699421485865


<ipython-input-643-aabf7bbada85>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data['ds'] = pd.to_datetime(training_data['ds'])
<ipython-input-643-aabf7bbada85>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_data['ds'] = pd.to_datetime(future_data['ds'])
16:00:20 - cmdstanpy - INFO - Chain [1] start processing
16:00:20 - cmdstanpy - INFO - Chain [1] done processing


...

RSAFS employ_rate 4 0.016056373187319246


16:04:27 - cmdstanpy - INFO - Chain [1] start processing
16:04:27 - cmdstanpy - INFO - Chain [1] done processing
16:04:29 - cmdstanpy - INFO - Chain [1] start processing
16:04:29 - cmdstanpy - INFO - Chain [1] done processing
16:04:32 - cmdstanpy - INFO - Chain [1] start processing
16:04:32 - cmdstanpy - INFO - Chain [1] done processing
16:04:35 - cmdstanpy - INFO - Chain [1] start processing
16:04:35 - cmdstanpy - INFO - Chain [1] done processing
<ipython-input-643-aabf7bbada85>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data['ds'] = pd.to_datetime(training_data['ds'])
<ipython-input-643-aabf7bbada85>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

RSAFS indust_prod 4 0.007835754640182725


16:04:39 - cmdstanpy - INFO - Chain [1] start processing
16:04:40 - cmdstanpy - INFO - Chain [1] done processing
16:04:42 - cmdstanpy - INFO - Chain [1] start processing
16:04:42 - cmdstanpy - INFO - Chain [1] done processing
16:04:44 - cmdstanpy - INFO - Chain [1] start processing


KeyboardInterrupt: 